In [1]:
from ortools.linear_solver import pywraplp as OR
import itertools
import pandas as pd

In [2]:
tsp_6_node = pd.read_csv('tsp_6_node.csv', index_col=0).astype(int)
tsp_6_node.columns = tsp_6_node.columns.astype(int)

In [3]:
def two_matching(graph, cuts, integer=False):
    """A model to solve tsp problems with two-matching."""
    NODES = list(graph)                      # nodes
    cost = {}                                # costs
    for i in NODES:
        for j in NODES:
            if i < j:
                cost[(i,j)] = graph.at[i,j]
    EDGES = list(cost)                       # edges
    
    # define model
    m = OR.Solver('two_matching', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # decision variables
    if integer:
        x = {(i,j) : m.IntVar(0, 1, ('(%s,%s)' % (i,j))) for (i,j) in EDGES}
    else:
        x = {(i,j) : m.NumVar(0, 1, ('(%s,%s)' % (i,j))) for (i,j) in EDGES}
    
    # objective function
    m.Minimize(sum(cost[i,j]*x[i,j] for (i,j) in EDGES))
    
    # subject to: degree of every node is 2
    for k in NODES:
        m.Add(sum(x[i,j] for (i,j) in EDGES if i == k) + 
              sum(x[i,j] for (i,j) in EDGES if j == k) == 2)
    
    # subject to: provided cuts
    for S in cuts:
        m.Add(sum(x[i,j] for (i,j) in EDGES if (i in S and j not in S) or 
                                               (j in S and i not in S)) >= 2)
        
    return m,x

In [4]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [5]:
m,x = two_matching(tsp_6_node, [])
solve(m)

Solution:
Objective value = 6.0
(1,2) : 1.0
(1,3) : 1.0
(1,4) : 0.0
(1,5) : 0.0
(1,6) : 0.0
(2,3) : 1.0
(2,4) : 0.0
(2,5) : 0.0
(2,6) : 0.0
(3,4) : 0.0
(3,5) : 0.0
(3,6) : 0.0
(4,5) : 1.0
(4,6) : 1.0
(5,6) : 1.0


In [6]:
m,x = two_matching(tsp_6_node, [[1,2,3]])
solve(m)

Solution:
Objective value = 14.0
(1,2) : 0.0
(1,3) : 1.0
(1,4) : 1.0
(1,5) : 0.0
(1,6) : 0.0
(2,3) : 1.0
(2,4) : 0.0
(2,5) : 1.0
(2,6) : 0.0
(3,4) : 0.0
(3,5) : 0.0
(3,6) : 0.0
(4,5) : 0.0
(4,6) : 1.0
(5,6) : 1.0
